# Dataset Updates 2023
Script to update final products with new data from BBay, Kodiak and PWS using updated NHD Waterbody/WBD Huc12s downloaded in May 2023. Filter old PWS from existing CI/CR outputs and update with new records.
Overwrite existing datasets on T: and update data stored on Google Drive.
Note updates in [AKKSF GIS Products worksheet](https://docs.google.com/spreadsheets/d/1TJrNIwr14HJ4QV6-fB4ubmi0BoTNCG7UxoK-s--IsJM/edit#gid=0)

## Data Sets to be updated:
| GIS process or output                                               | Output name                       | Output Location            |
|---------------------------------------------------------------------|-----------------------------------|----------------------------|
| table of catchment/watershed covariates for AWC Huc12s              | AKSSF_awcHuc12s_SpLs.csv          | In AKSSF repository        |
| polygon fc of AWC Huc12s                                            | AKSSF_awcHuc12s                   | AKSSF_Hydrography.gdb on T: |
| point fc of AWC Huc12s outlets                                      | AKSSF_awcHuc12_outlet_cats_points | AKSSF_Hydrography.gdb on T: |
| polygon fc of AWC Huc12 Watersheds                                  | AKSSF_AwcHuc12_Watersheds_Merge   | AKSSF_Hydrography.gdb on T: |
| polygon fc of AWC Huc12 outlet catchments                           | AKSSF_awcHuc12_outlet_cats_merge  | AKSSF_Hydrography.gdb on T: |
| AWC species and lstage for HUC12s as catchment outlet feature class | AKSSF_awcHuc12_cats_awc_spJoin    | AKSSF_Hydrography.gdb on T: |




In [1]:
#Import Modules
import arcpy, os

def append_value(dict_obj, key, value):
    #Check if key exist in dict or not
    if key in dict_obj:
        #Key exist in dict.
        #Check if type of value of key is list or not
        if not isinstance(dict_obj[key], list):
            #If type is not list then make it list
            dict_obj[key] = [dict_obj[key]]
        #Append the value in list
        dict_obj[key].append(value)
    else:
        #As key is not in dict,
        #so, add key-value pair
        dict_obj[key] = value
origOutDir = r'D:\\GIS\\AKSSF_awcHuc12_cv_Final'
origOutGdb = os.path.join(origOutDir, 'AKSSF_awcHuc12_cv_Final.gdb')
netGdb = r"T:\\Aquatic\\AKSSF\\AKSSF_Hydrography.gdb"
updDir = r'D:\\GIS\\outputs_20230518'
updGdb =r'D:\\GIS\\outputs_20230518\\outputs_20230518.gdb'
fcs = ['AKSSF_awcHuc12s','AKSSF_awcHuc12_outlet_cats_points','AKSSF_AwcHuc12_Watersheds_Merge','AKSSF_awcHuc12_outlet_cats_merge','AKSSF_awcHuc12_cats_awc_spJoin','AKSSF_awcHuc12_Covariates'] #Covariates is a db Table

updDict ={}

for fc in fcs:
    netfc = os.path.join(netGdb,fc)
    oldfc = os.path.join(origOutGdb,fc)
    if fc == 'AKSSF_awcHuc12_outlet_cats_points':
        updfc = os.path.join(updGdb,fc+'_SpLs')
    else:updfc = os.path.join(updGdb,fc)
    append_value(updDict,fc,[oldfc,updfc,netfc])

print(f'Dictionaries created')
print(updDict)

Dictionaries created
{'AKSSF_awcHuc12s': ['D:\\\\GIS\\\\AKSSF_awcHuc12_cv_Final\\AKSSF_awcHuc12_cv_Final.gdb\\AKSSF_awcHuc12s', 'D:\\\\GIS\\\\outputs_20230518\\\\outputs_20230518.gdb\\AKSSF_awcHuc12s', 'T:\\\\Aquatic\\\\AKSSF\\\\AKSSF_Hydrography.gdb\\AKSSF_awcHuc12s'], 'AKSSF_awcHuc12_outlet_cats_points': ['D:\\\\GIS\\\\AKSSF_awcHuc12_cv_Final\\AKSSF_awcHuc12_cv_Final.gdb\\AKSSF_awcHuc12_outlet_cats_points', 'D:\\\\GIS\\\\outputs_20230518\\\\outputs_20230518.gdb\\AKSSF_awcHuc12_outlet_cats_points_SpLs', 'T:\\\\Aquatic\\\\AKSSF\\\\AKSSF_Hydrography.gdb\\AKSSF_awcHuc12_outlet_cats_points'], 'AKSSF_AwcHuc12_Watersheds_Merge': ['D:\\\\GIS\\\\AKSSF_awcHuc12_cv_Final\\AKSSF_awcHuc12_cv_Final.gdb\\AKSSF_AwcHuc12_Watersheds_Merge', 'D:\\\\GIS\\\\outputs_20230518\\\\outputs_20230518.gdb\\AKSSF_AwcHuc12_Watersheds_Merge', 'T:\\\\Aquatic\\\\AKSSF\\\\AKSSF_Hydrography.gdb\\AKSSF_AwcHuc12_Watersheds_Merge'], 'AKSSF_awcHuc12_outlet_cats_merge': ['D:\\\\GIS\\\\AKSSF_awcHuc12_cv_Final\\AKSSF_awcHuc12

In [33]:
#Add fields that are missing from updated datasets
wtdmissfields = {'wtdAreaSqKm': ['Watershed Area Square Kilometers','DOUBLE'], 'NHDPlusID':['NHDPlusID','Double']}
upwtds = updDict['AKSSF_AwcHuc12_Watersheds_Merge'][1]
upwtdsfields = [f.name for f in arcpy.ListFields(upwtds)]
for k,v in wtdmissfields.items():
    field = k
    alias = v[0]
    type = v[1]
    if field not in upwtdsfields:
        print(f'Adding field {field}')
        arcpy.AddField_management(upwtds,field_name=field,field_type=type, field_alias=alias)
    else:
        print(f'Field {field} already exists')

# Create an update cursor to calculate and update the area field
with arcpy.da.UpdateCursor(upwtds, ["SHAPE@AREA", "wtdAreaSqKm"]) as cursor:
    for row in cursor:
        if row[1] is None:
            # Get the area in square meters
            area_sqm = row[0]
            # Calculate the area in square kilometers
            area_sqkm = area_sqm / 1000000
            # Update the area_km2 field with the calculated value
            row[1] = area_sqkm
            # Update the feature
            cursor.updateRow(row)
        else:
            continue

# Create an update cursor to calculate and update the area field
with arcpy.da.SearchCursor(upwtds, ["SHAPE@AREA", "cat_ID_con", "wtdAreaSqKm"]) as cursor:
    for row in cursor:
        print(f'{row[1]} has shape area {row[0]} = {row[2]}sqKm')


Field wtdAreaSqKm already exists
Field NHDPlusID already exists
Bristol_Bay_1003859 has shape area 39975900.0 = 39.9759sqKm
Bristol_Bay_1004129 has shape area 45335900.0 = 45.3359sqKm
Bristol_Bay_1006868 has shape area 35790300.0 = 35.7903sqKm
Bristol_Bay_1007408 has shape area 42596800.0 = 42.5968sqKm
Bristol_Bay_1008168 has shape area 45597900.0 = 45.5979sqKm
Bristol_Bay_1008527 has shape area 69569500.0 = 69.5695sqKm
Bristol_Bay_1008597 has shape area 117894900.0 = 117.8949sqKm
Bristol_Bay_1013111 has shape area 56216600.0 = 56.2166sqKm
Bristol_Bay_1013306 has shape area 99641800.0 = 99.6418sqKm
Bristol_Bay_1013765 has shape area 49397900.0 = 49.3979sqKm
Bristol_Bay_1014251 has shape area 142753800.0 = 142.7538sqKm
Bristol_Bay_1014341 has shape area 121891400.0 = 121.8914sqKm
Bristol_Bay_1014516 has shape area 60913000.0 = 60.913sqKm
Bristol_Bay_1014641 has shape area 256457100.0 = 256.4571sqKm
Bristol_Bay_1014786 has shape area 81889800.0 = 81.8898sqKm
Bristol_Bay_1014821 has shape

In [46]:
# Delete ftype field from watersheds merge
arcpy.DeleteField_management(updDict['AKSSF_AwcHuc12_Watersheds_Merge'][1],'ftype')

<Result 'D:\\\\GIS\\\\outputs_20230518\\\\outputs_20230518.gdb\\AKSSF_AwcHuc12_Watersheds_Merge'>

In [2]:
c=0
for item in updDict.items():
    print(c, item)
    c+=1

0 ('AKSSF_awcHuc12s', ['D:\\\\GIS\\\\AKSSF_awcHuc12_cv_Final\\AKSSF_awcHuc12_cv_Final.gdb\\AKSSF_awcHuc12s', 'D:\\\\GIS\\\\outputs_20230518\\\\outputs_20230518.gdb\\AKSSF_awcHuc12s', 'T:\\\\Aquatic\\\\AKSSF\\\\AKSSF_Hydrography.gdb\\AKSSF_awcHuc12s'])
1 ('AKSSF_awcHuc12_outlet_cats_points', ['D:\\\\GIS\\\\AKSSF_awcHuc12_cv_Final\\AKSSF_awcHuc12_cv_Final.gdb\\AKSSF_awcHuc12_outlet_cats_points', 'D:\\\\GIS\\\\outputs_20230518\\\\outputs_20230518.gdb\\AKSSF_awcHuc12_outlet_cats_points_SpLs', 'T:\\\\Aquatic\\\\AKSSF\\\\AKSSF_Hydrography.gdb\\AKSSF_awcHuc12_outlet_cats_points'])
2 ('AKSSF_AwcHuc12_Watersheds_Merge', ['D:\\\\GIS\\\\AKSSF_awcHuc12_cv_Final\\AKSSF_awcHuc12_cv_Final.gdb\\AKSSF_AwcHuc12_Watersheds_Merge', 'D:\\\\GIS\\\\outputs_20230518\\\\outputs_20230518.gdb\\AKSSF_AwcHuc12_Watersheds_Merge', 'T:\\\\Aquatic\\\\AKSSF\\\\AKSSF_Hydrography.gdb\\AKSSF_AwcHuc12_Watersheds_Merge'])
3 ('AKSSF_awcHuc12_outlet_cats_merge', ['D:\\\\GIS\\\\AKSSF_awcHuc12_cv_Final\\AKSSF_awcHuc12_cv_Final.

In [66]:
arcpy.AddField_management(updDict['AKSSF_awcHuc12_Covariates'][1],'NHDPlusID','DOUBLE')
arcpy.DeleteField_management(updDict['AKSSF_awcHuc12_Covariates'][1],'ftype')

<Result 'D:\\\\GIS\\\\outputs_20230518\\\\outputs_20230518.gdb\\AKSSF_awcHuc12_Covariates'>

In [54]:
#Add missing fields to outlet cats merge (mostly NHDPLus Fields) and change gridcode to Gridcode
arcpy.AddFields_management(updDict['AKSSF_awcHuc12_outlet_cats_merge'][1],[['NHDPlusID'],
                           ['SourceFC','TEXT'],
#                           ['GridCode','DOUBLE'],
                           ['AreaSqKm','DOUBLE'],
                           ['VPUID','TEXT'],
                           ['TotDASqKm','DOUBLE'],
                          ['region','TEXT']])
#arcpy.CalculateField_management(updDict['AKSSF_awcHuc12_outlet_cats_merge'][1],'Gridcode',"!gridcode!")
arcpy.CalculateField_management(updDict['AKSSF_awcHuc12_outlet_cats_merge'][1],'region',expression="!cat_ID_con!.rsplit('_',1)[0]",)


<Result 'D:\\\\GIS\\\\outputs_20230518\\\\outputs_20230518.gdb\\AKSSF_awcHuc12_outlet_cats_merge'>

In [73]:
#Add region field to cats sp_jn

arcpy.AddField_management(updDict['AKSSF_awcHuc12_cats_awc_spJoin'][1],'region','TEXT')
arcpy.CalculateField_management(updDict['AKSSF_awcHuc12_cats_awc_spJoin'][1],'region',expression="!cat_ID_con!.rsplit('_',1)[0]",)
arcpy.AddField_management(updDict['AKSSF_awcHuc12_cats_awc_spJoin'][0],'region','TEXT')
arcpy.CalculateField_management(updDict['AKSSF_awcHuc12_cats_awc_spJoin'][0],'region',expression="!cat_ID_con!.rsplit('_',1)[0]",)


<Result 'D:\\\\GIS\\\\AKSSF_awcHuc12_cv_Final\\AKSSF_awcHuc12_cv_Final.gdb\\AKSSF_awcHuc12_cats_awc_spJoin'>

## Update Network and Local copies
If running again compare fields and make sure they match, may need to add/Delete some fields before running insert cursor

In [50]:
updfs = {}
for k,v in updDict.items():
    locFeature = v[0]
    netFeature = v[2]
    upFeature = v[1]
    if k != 'AKSSF_awcHuc12_Covariates':
        insFields = ['SHAPE@'] + updFields
    else:
        insFields = updFields
    fcount = 0
    dcount = 0
    oriFields = [f.name for f in arcpy.ListFields(locFeature)]
    updFields = [f.name for f in arcpy.ListFields(upFeature)]
    netFields = [f.name for f in arcpy.ListFields(netFeature)]
    print(f'{k}')
    # # Find the common fields between the two feature classes
    # #common_fields = set(oriFields).intersection(updFields)
    # if oriFields == updFields:
    #     print(locFeature)
    #     print(f'Ori Fields {oriFields}\n Upd Fields {updFields}\n {"-"* 50}')
    #     print(f'Fields match!\n {"-"* 50}')
    # else:
    #     print(locFeature)
    #     missing_elements = list(set(oriFields) ^ set(updFields))
    #     print(f'Ori Fields {oriFields}\n Upd Fields {updFields}\n {"-"* 50}')
    #     print(f'Fields do not match\n Missing Fields = {missing_elements}\n {"-"* 50}')
    #
    # if netFields == updFields:
    #     print(locFeature)
    #     print(f'Network Fields {netFields}\n Upd Fields {updFields}\n {"-"* 50}')
    #     print(f'Fields match!\n {"-"* 50}')
    # else:
    #     print(locFeature)
    #     missing_elements = list(set(oriFields) ^ set(netFields))
    #     print(f'Ori Fields {oriFields}\n Upd Fields {netFields}\n {"-"* 50}')
    #     print(f'Fields do not match\n Missing Fields = {missing_elements}\n {"-"* 50}')

    try:
        with arcpy.da.UpdateCursor(netFeature,"cat_ID_con") as fccur:
            for row in fccur:
                if row[0].startswith(("Prince","Bristol","Kodiak")):
                    print (f'Deleting {row} from {netFeature}')
                    fccur.deleteRow()
                    dcount += 1
                else:
                    pass
    except Exception as e:
        print(f"Error updating {netFeature}: {e}")

    try:
        # Create an insert cursor for the target feature class
        with arcpy.da.SearchCursor(upFeature, insFields) as scur:
            # Open an insert cursor on the feature class to update
            with arcpy.da.InsertCursor(netFeature, insFields) as insert_cursor:
                # Iterate through the rows of the local feature class
                for row in scur:
                    # Create a new row in the network feature class and write the geometry and attribute values
                    insert_cursor.insertRow(row)
                    fcount += 1
    except Exception as e:
        print(f"Error updating {netFeature}: {e}")
    append_value(updfs,netFeature,[dcount,fcount])

print('Features updated\n')

for k,v in updfs.items():
    print(f'{v[1]} records deleted\n'
          f'{v[0]} records added to {k}')

AKSSF_awcHuc12s
Deleting ['Prince_William_Sound_74542'] from T:\\Aquatic\\AKSSF\\AKSSF_Hydrography.gdb\AKSSF_awcHuc12s
Deleting ['Prince_William_Sound_37935'] from T:\\Aquatic\\AKSSF\\AKSSF_Hydrography.gdb\AKSSF_awcHuc12s
Deleting ['Prince_William_Sound_78912'] from T:\\Aquatic\\AKSSF\\AKSSF_Hydrography.gdb\AKSSF_awcHuc12s
Deleting ['Prince_William_Sound_43793'] from T:\\Aquatic\\AKSSF\\AKSSF_Hydrography.gdb\AKSSF_awcHuc12s
Deleting ['Prince_William_Sound_44113'] from T:\\Aquatic\\AKSSF\\AKSSF_Hydrography.gdb\AKSSF_awcHuc12s
Deleting ['Prince_William_Sound_39255'] from T:\\Aquatic\\AKSSF\\AKSSF_Hydrography.gdb\AKSSF_awcHuc12s
Deleting ['Prince_William_Sound_75002'] from T:\\Aquatic\\AKSSF\\AKSSF_Hydrography.gdb\AKSSF_awcHuc12s
Deleting ['Prince_William_Sound_45863'] from T:\\Aquatic\\AKSSF\\AKSSF_Hydrography.gdb\AKSSF_awcHuc12s
Deleting ['Prince_William_Sound_45835'] from T:\\Aquatic\\AKSSF\\AKSSF_Hydrography.gdb\AKSSF_awcHuc12s
Deleting ['Prince_William_Sound_62232'] from T:\\Aquatic\

In [32]:
list1 = [f.name for f in arcpy.ListFields(updDict['AKSSF_awcHuc12_Covariates'][2])]
list2 = [f.name for f in arcpy.ListFields(updDict['AKSSF_awcHuc12_Covariates'][1])]
missing_dict = {f"list1": [value for value in list1 if value not in list2],
                f"list2": [value for value in list2 if value not in list1]}

print(missing_dict)
print(updDict['AKSSF_AwcHuc12_Watersheds_Merge'][1])

{'list1': [], 'list2': []}
D:\\GIS\\outputs_20230518\\outputs_20230518.gdb\AKSSF_AwcHuc12_Watersheds_Merge


# Delete HUC12 fc from T: and replace with updated version


In [116]:
arcpy.env.overwriteOutput = True
splsTbl = arcpy.TableToTable_conversion(updDict['AKSSF_awcHuc12_Covariates'][0], origOutGdb,'AKSSF_awcHuc12s_SpLs')
arcpy.JoinField_management(splsTbl,'cat_ID_con',updDict['AKSSF_awcHuc12_cats_awc_spJoin'][0],'cat_ID_con')
arcpy.JoinField_management(splsTbl,'cat_ID_con',updDict['AKSSF_awcHuc12_outlet_cats_merge'][0],'cat_ID_con')
dropfields = ['Shape_Length_1', 'Shape_Area_1','MERGE_SRC_1', 'Join_Count', 'TARGET_FID', 'cat_ID_con_1',
              'NHDPlusID_1', 'SourceFC', 'MERGE_SRC_12', 'Shape_Length_12', 'Shape_Area_12', 'HUC12_1',
              'region_1','cat_ID_con_12']
arcpy.DeleteField_management(splsTbl,drop_field=dropfields)
splsFields = [f.name for f in arcpy.ListFields(splsTbl)]
print(splsFields)

['OBJECTID', 'ORIG_FID', 'MERGE_SRC', 'cat_ID_con', 'cat_ID', 'cat_ID_txt', 'NHDPlusID', 'Shape_Length', 'Shape_Area', 'AwcHuc12_non_north_area', 'AwcHuc12_north_area', 'AwcHuc12_wtd_north_per', 'AwcHuc12_cat_elev_ZONE_CODE', 'AwcHuc12_cat_elev_COUNT', 'AwcHuc12_cat_elev_AREA', 'AwcHuc12_cat_elev_MIN', 'AwcHuc12_cat_elev_MAX', 'AwcHuc12_cat_elev_RANGE', 'AwcHuc12_cat_elev_MEAN', 'AwcHuc12_cat_elev_STD', 'AwcHuc12_cat_elev_SUM', 'AwcHuc12_cat_elev_VARIETY', 'AwcHuc12_cat_elev_MAJORITY', 'AwcHuc12_cat_elev_MINORITY', 'AwcHuc12_cat_elev_MEDIAN', 'AwcHuc12_cat_elev_PCT90', 'AwcHuc12_wtd_elev_MIN', 'AwcHuc12_wtd_elev_MAX', 'AwcHuc12_wtd_elev_MEAN', 'AwcHuc12_wtd_elev_STD', 'AwcHuc12_wtd_slope_MIN', 'AwcHuc12_wtd_slope_MAX', 'AwcHuc12_wtd_slope_MEAN', 'AwcHuc12_wtd_slope_STD', 'AwcHuc12_cat_slope_ZONE_CODE', 'AwcHuc12_cat_slope_COUNT', 'AwcHuc12_cat_slope_AREA', 'AwcHuc12_cat_slope_MIN', 'AwcHuc12_cat_slope_MAX', 'AwcHuc12_cat_slope_RANGE', 'AwcHuc12_cat_slope_MEAN', 'AwcHuc12_cat_slope_STD'

In [51]:
# Had to change input to actual path because conversion was writing 0 values for some fields?

repodir = r'C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers'
arcpy.env.overwriteOutput = True
# Export Covariate csvs
arcpy.conversion.TableToTable("D:\\GIS\\AKSSF_awcHuc12_cv_Final\\AKSSF_awcHuc12_cv_Final.gdb\\AKSSF_AwcHuc12_Covariates", repodir, 'AKSSF_awcHuc12s_Covariates.csv')
arcpy.conversion.TableToTable("D:\\GIS\\AKSSF_awcHuc12_cv_Final\\AKSSF_awcHuc12_cv_Final.gdb\\AKSSF_awcHuc12s_SpLs", repodir, 'AKSSF_awcHuc12s_SpLs.csv')

print('Export all covariates csv complete')


Export all covariates csv complete


## Compare old Covariates tables to new outputs

In [1]:
import arcpy
import pandas as pd

# Specify the path and name of the first table
newTable = r'T:\\Aquatic\\AKSSF\\AKSSF_Hydrography.gdb\\AKSSF_awcHuc12_Covariates'

# Specify the path and name of the second table
oldTable = r"U:\\GIS\\AKSSF_Backup\\AKSSF_awcHuc12_cv_Final\\AKSSF_awcHuc12_cv_Final.gdb\\AKSSF_AwcHuc12_Covariates"
#oldTable = r"D:\\GIS\\AKSSF_awcHuc12_cv_Final\\AKSSF_awcHuc12_cv_Final.gdb\\AKSSF_AwcHuc12_Covariates"

# Specify the common identifier field
common_field = 'cat_ID_con'

# Get the fields common to both tables
fields_table1 = [field.name for field in arcpy.ListFields(newTable)]
fields_table2 = [field.name for field in arcpy.ListFields(oldTable)]
common_fields = sorted(list(set(fields_table1) & set(fields_table2)))

# Create dictionaries to store the differences and missing fields
diff_Details = {}
diff_Details2 = {}
missFlds = {}

# Function to add key, value pairs to dictionary
def append_value(dict_obj, key, value):
    # Check if key exist in dict or not
    if key in dict_obj:
        # Key exists in dict.
        # Check if type of value of key is list or not
        if not isinstance(dict_obj[key], list):
            # If type is not list then make it list
            dict_obj[key] = [dict_obj[key]]
        # Append the value in list
        dict_obj[key].append(value)
    else:
        # As key is not in dict,
        # so, add key-value pair
        dict_obj[key] = value

# Function to sort values alphabetically for "sp_ls" field
def sort_sp_ls(values):
    if isinstance(values, str):
        # Split the string values and sort alphabetically
        sorted_values = ', '.join(sorted(values.split(', ')))
        return sorted_values
    return values

# Open a search cursor on the first table using the common fields
with arcpy.da.SearchCursor(newTable, [common_field] + common_fields) as cursor1:
    # Iterate through each row in the first table
    for row1 in cursor1:
        cat_ID_con = str(row1[0])
        field_values1 = row1[1:]

        # Open a search cursor on the second table using the common fields
        with arcpy.da.SearchCursor(oldTable, [common_field] + common_fields) as cursor2:
            # Find the corresponding row in the second table based on the common identifier
            for row2 in cursor2:
                if str(row2[0]) == cat_ID_con:
                    field_values2 = row2[1:]

                    # Compare all fields between the two tables
                    for i, field_name in enumerate(common_fields):
                        value1 = field_values1[i]
                        value2 = field_values2[i]
                        source_table = 'newTable' if i < len(field_values1) else 'oldTable'

                        # Sort values alphabetically for "sp_ls" field
                        if field_name == 'sp_ls':
                            value1 = sort_sp_ls(value1)
                            value2 = sort_sp_ls(value2)

                        # Check if both values are not None
                        if value1 is not None and value2 is not None:
                            # Check if the field is a string
                            if isinstance(value1, str) or isinstance(value2, str):
                                # Store the string values instead of calculating the difference
                                if value1 != value2:
                                    append_value(diff_Details, cat_ID_con, (field_name, value1, value2))
                            else:
                                # Calculate the difference between numeric values
                                if isinstance(value1, tuple) and isinstance(value2, tuple):
                                    diff_value = tuple(v2 - v1 for v1, v2 in zip(value1, value2))
                                else:
                                    diff_value = value2 - value1
                                if diff_value != 0:
                                    append_value(diff_Details, cat_ID_con, (field_name, diff_value))
                                    if isinstance(diff_value, (int, float)) and abs(diff_value) > 1:
                                        append_value(diff_Details2, cat_ID_con, (field_name, diff_value))
                        else:
                            # Handle the case when one of the values is None (The should only be happening for lake/pond related fields)
                            if value1 is None and value2 is not None:
                                diff_value = value2 * -1
                                append_value(diff_Details, cat_ID_con, (field_name, diff_value))
                            elif value1 is not None and value2 is None:
                                diff_value = value1
                                append_value(diff_Details, cat_ID_con, (field_name, diff_value))

# Store the missing fields in the missFlds dictionary
missing_fields_table1 = sorted(list(set(fields_table1) - set(common_fields)))
missing_fields_table2 = sorted(list(set(fields_table2) - set(common_fields)))

if missing_fields_table1:
    missFlds[newTable] = missing_fields_table1

if missing_fields_table2:
    missFlds[oldTable] = missing_fields_table2

print(f'{len(diff_Details)} items in diff_Details')
print(f'{len(diff_Details2)} items in diff_Details2')


0 items in diff_Details
0 items in diff_Details2


In [53]:

# Print the differences for vals > 1
for key, value in diff_Details.items():
    print(f"{key}: {value}\n")


Prince_William_Sound_685: [('AwcHuc12_wtd_lake_area_sqm', -0.004315698512073141), ('AwcHuc12_wtd_lake_per', -8.296229356830054e-07), ('MERGE_SRC', 'D:\\GIS\\outputs_20230518\\outputs_20230518.gdb\\Prince_William_Sound_Watersheds\\Prince_William_Sound_wtd_685', 'D:\\GIS\\AKSSF_awcHuc12_cv_Final\\AKSSF_awcHuc12_cv_Final.gdb\\Prince_William_Sound_Watersheds\\Prince_William_Sound_wtd_685'), ('OBJECTID', -2414)]

Prince_William_Sound_3038: [('AwcHuc12_wtd_lake_area_sqm', 31113.025698131903), ('AwcHuc12_wtd_lake_per', 0.0959727616187371), ('MERGE_SRC', 'D:\\GIS\\outputs_20230518\\outputs_20230518.gdb\\Prince_William_Sound_Watersheds\\Prince_William_Sound_wtd_3038', 'D:\\GIS\\AKSSF_awcHuc12_cv_Final\\AKSSF_awcHuc12_cv_Final.gdb\\Prince_William_Sound_Watersheds\\Prince_William_Sound_wtd_3038'), ('OBJECTID', -2414)]

Prince_William_Sound_13227: [('MERGE_SRC', 'D:\\GIS\\outputs_20230518\\outputs_20230518.gdb\\Prince_William_Sound_Watersheds\\Prince_William_Sound_wtd_13227', 'D:\\GIS\\AKSSF_awcHu

In [ ]:
strOrdTbl  = r''
with arcpy.da.SearchCursor(strOrdTbl,['','']) as cur:12